In [ ]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
import torch
from torch import nn
from torch import optim
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import torch.utils.data as data_utils
from PIL import Image, ImageFile
import json
from torch.optim import lr_scheduler
import time
import os
import argparse
import copy
import pandas as pd
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import sklearn
from sklearn.metrics import cohen_kappa_score, accuracy_score
import time
from tqdm import tqdm_notebook

In [ ]:
import seaborn as sns

In [ ]:
!kaggle competitions download -c aptos2019-blindness-detection

In [ ]:
!unzip aptos2019-blindness-detection.zip -d aptos2019

In [ ]:
train_csv = pd.read_csv('/content/aptos2019/train.csv')
test_csv = pd.read_csv('/content/aptos2019/test.csv')

In [ ]:
print('Train Size = {}'.format(len(train_csv)))
print('Public Test Size = {}'.format(len(test_csv)))

In [ ]:
train_csv.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

counts = train_csv['diagnosis'].value_counts()
class_list = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferate']

for i, x in enumerate(class_list):
    counts[x] = counts.pop(i)

plt.figure(figsize=(10, 5))
sns.barplot(x=counts.index, y=counts.values, alpha=0.8, palette='bright')
plt.title('Distribution of Output Classes')
plt.xlabel('Target Classes', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

base_dir = '/content/aptos2019'

train_imgs = os.listdir(os.path.join(base_dir, 'train_images'))

fig = plt.figure(figsize=(30, 6))
for idx, img in enumerate(np.random.choice(train_imgs, 16)):
    ax = fig.add_subplot(2, 16 // 2, idx + 1, xticks=[], yticks=[])
    im = Image.open(os.path.join(base_dir, "train_images", img))
    plt.imshow(im)
    lab = train_csv.loc[train_csv['id_code'] == img.split('.')[0], 'diagnosis'].values[0]
    ax.set_title('Severity: %s' % lab)

plt.show()

In [ ]:
fig = plt.figure(figsize=(30, 6))
test_imgs = os.listdir(base_dir+"/test_images")
for idx, img in enumerate(np.random.choice(test_imgs, 16)):
    ax = fig.add_subplot(2, 16//2, idx+1, xticks=[], yticks=[])
    im = Image.open(base_dir+"/test_images/" + img)
    plt.imshow(im)

In [ ]:
class CreateDataset(Dataset):
    def __init__(self, df_data, data_dir = '../input/', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name+'.png')
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [ ]:
train_path = "/content/aptos2019/train_images"
test_path = "/content/aptos2019/test.csv"

In [ ]:
train_data = CreateDataset(df_data=train_csv, data_dir=train_path, transform=train_transforms)
test_data = CreateDataset(df_data=test_csv, data_dir=test_path, transform=test_transforms)

In [ ]:
valid_size = 0.2
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64,sampler=train_sampler)
validloader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=valid_sampler)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
print(f"training examples contain : {len(train_data)}")
print(f"testing examples contain : {len(test_data)}")

In [ ]:
images, labels = next(iter(trainloader))

print(f"Image shape : {images.shape}")
print(f"Label shape : {labels.shape}")

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
import torchvision
import matplotlib.pyplot as plt
import numpy as np
grid = torchvision.utils.make_grid(images, nrow=30, padding=7)
grid = grid.cpu()
plt.figure(figsize=(20, 20))
plt.imshow(np.transpose(grid, (1, 2, 0)))
print('labels:', labels)

In [ ]:
class_names = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']

images, labels = next(iter(trainloader))
out = torchvision.utils.make_grid(images)
imshow(out, title=[class_names[x] for x in labels])

In [ ]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:

import os
import shutil
import random

train = '/content/aptos2019/train_images'
test = '/content/aptos2019/test_images'
os.makedirs('test' , exist_ok=True)
os.makedirs('train' , exist_ok=True)
os.makedirs('val' , exist_ok=True)

t_df = pd.read_csv('/content/aptos2019/train.csv')

train_df = t_df.sample(frac = 0.80)
test_df = t_df.drop(train_df.index)
test_df.reset_index(drop=True, inplace=True)

val_df = test_df.sample(frac = 0.45)
test_df = test_df.drop(val_df.index)
test_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

train_df.sort_values(by='diagnosis', inplace=True)
train_df.reset_index(drop=True, inplace=True)

test_df.sort_values(by='diagnosis', inplace=True)
test_df.reset_index(drop=True, inplace=True)

val_df.sort_values(by='diagnosis', inplace=True)
val_df.reset_index(drop=True, inplace=True)

In [ ]:
test_df['diagnosis'].value_counts().plot(kind='pie' , labels = list(test_df['diagnosis'].value_counts()))

In [ ]:
val_df['diagnosis'].value_counts().plot(kind='pie' , labels = list(val_df['diagnosis'].value_counts()))

In [ ]:
for i in range(5):
  imgs = train_df[train_df['diagnosis'] == i]['id_code'].to_list()
  os.makedirs(os.path.join('train', str(i)), exist_ok=True)
  for j in imgs:
    shutil.copy(os.path.join(train, j+'.png'), os.path.join(f'train/{i}', str(j)+'.png'))

In [ ]:
for i in range(5):
  imgs = test_df[test_df['diagnosis'] == i]['id_code'].to_list()
  os.makedirs(os.path.join('test', str(i)), exist_ok=True)
  for j in imgs:
    shutil.copy(os.path.join(train, j+'.png'), os.path.join(f'test/{i}', str(j)+'.png'))

In [ ]:
for i in range(5):
  imgs = val_df[val_df['diagnosis'] == i]['id_code'].to_list()
  os.makedirs(os.path.join('val', str(i)), exist_ok=True)
  for j in imgs:
    shutil.copy(os.path.join(train, j+'.png'), os.path.join(f'val/{i}', str(j)+'.png'))

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [ ]:
!pip install pretrainedmodels

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from pretrainedmodels import inceptionresnetv2

model = inceptionresnetv2(num_classes=1000, pretrained='imagenet')

layers = list(model.children())
for layer in layers[:-50]:
    for param in layer.parameters():
        param.requires_grad = False

num_classes = 5
in_features = model.last_linear.in_features
model.last_linear = nn.Linear(in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00001)
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomVerticalFlip(p=0.4),
    transforms.RandomRotation(degrees=(0, 10)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(root='/content/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root='/content/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

best_val_accuracy = 0.0
best_model_path = "best_model.pth"  # Path to save the best model

num_epochs = 10
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Training...")

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        if (batch_idx + 1) % 10 == 0:  # Print every 10 batches
            train_accuracy = 100 * correct_train / total_train
            print(f"Batch [{batch_idx + 1}/{len(train_loader)}] - "
                  f"Loss: {loss.item():.4f}, "
                  f"Train Accuracy: {train_accuracy:.2f}%")

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Validating...")

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)
        print(f"Best model saved with validation accuracy: {best_val_accuracy:.2f}%")

    print(f"Epoch [{epoch + 1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, "
          f"Val Accuracy: {val_accuracy:.2f}%, "
          f"Time: {time.time() - start_time:.2f} seconds\n")

print("Training complete. Best model saved at:", best_model_path)


In [ ]:
torch.save(model, 'model.pth')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pretrainedmodels
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score

model = pretrainedmodels.__dict__['inceptionresnetv2'](num_classes=1000, pretrained='imagenet')
num_classes = 5

in_features = model.last_linear.in_features
model.last_linear = nn.Linear(in_features, num_classes)

model.load_state_dict(torch.load('best_model.pth'))
model.eval()


transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dir = '/content/test'
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = test_loss / total
    accuracy = correct / total

    return avg_loss, accuracy, all_preds, all_labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
test_loss, test_accuracy, all_preds, all_labels = evaluate(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")

precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
kappa_score = cohen_kappa_score(all_labels, all_preds)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Cohen's Kappa Score: {kappa_score:.4f}")